# Residue Standardization

## What is the goal of this notebook? 

This is the final step to create our curated ensemble. Now that the chains have been standardized, we will ensure that the residue numbers are consistent among all structures. This is done by performing a multiple sequence alignment (MSA) for all the sequences in our ensemble and then generating a new numbering system based on the MSA. 

Choosing a new numbering system that is consistent for all structures requires dealing with gaps in the MSA. Gaps in the alignment could be due to a difference in sequence length or to residues not being resolved in the structure. When we are assigning residue numbers, if we encounter a position in which a gap is present in at least 75% of the sequences, we stop counting and identify this position with the last number we counted plus a letter (in increasing alphabetical order, in case there are multiple residues). The letter is added in the "pdbx_PDB_ins_code" column.     

Since the final numbering depends heavily on the MSA, users must inspect the MSA generated. By default, PDBClean will use Muscle v.5.1 to create an alignment if users provide none. However, users can also provide their alignments. PDBClean will look for existing fasta files before running Muscle. 

This notebook continues with the TIM dataset curation and shows the default PDBClean usage. Advanced MSA manipulation will not be covered. However, we encourage users to compare different MSA methods and find the one best for their systems.   


>**NOTE 1:** For this tutorial, we will not use the whole ensemble we downloaded. We will use a subsample of only 6 structures. The next cells will create the new directory. Notice that we are choosing these 6 sctructures from the ones we downloaded in Step 0. We chose these ones to highlight some possible issues you may run into when running this script. 

>**NOTE 2:** This is the continuation of step 3


In [1]:
from PDBClean import pdbclean_io

In [2]:
PROJDIR="./TIM/"

In [3]:
pdbclean_io.check_project(projdir=PROJDIR, action='create', level='standard_ResidueID_bank')

### How to run `PDBClean_ResidueStandardization_CIF.py`

In the terminal, type:

> PDBClean_ResidueStandardization_CIF.py `{Input Directory}` `{Output Directory}`

Select the following choices when prompted in the on screen menus:  

`1) Perform multiple alignments to identify residues` -> 
`1) Show list of chains to be standardized` -> 
`4) Perform multiple alignments`

Wait for MUSCLE to finish running


`4) Save conversion template` \
`3) Perform residue number standardization` \
`Type QUIT`

For this tutorial, you can also run the following cell, which already includes the inputs for these options. 

In [4]:
! echo '1\n1\n4\n4\n3\nQUIT\n' | PDBClean_ResidueStandardization_CIF.py $PROJDIR/standard_ChainID_bank $PROJDIR/standard_ResidueID_bank

Reading: ./TIM//standard_ChainID_bank/2y62+00.cif  (1 of 6)
Reading: ./TIM//standard_ChainID_bank/1ag1+00.cif  (2 of 6)
Reading: ./TIM//standard_ChainID_bank/1aw1+04.cif  (3 of 6)
Reading: ./TIM//standard_ChainID_bank/1aw1+02.cif  (4 of 6)
Reading: ./TIM//standard_ChainID_bank/1aw1+03.cif  (5 of 6)
Reading: ./TIM//standard_ChainID_bank/1aw1+01.cif  (6 of 6)
PDBClean Residue Number Standardization Menu
    After checking all structures are loaded, select option 1 to proceed:
    1) Proceed to multiple alignment menu
Option Number:     Perform multiple alignments to identify residues
    1) Show list of chains to be standardized
    2) Remove chain IDs from list of chains to be standardized
    3) Input file of chain IDs to remove from list of chains to be standardized
    4) Perform multiple alignments
Option Number: A
B
    Perform multiple alignments to identify residues
    1) Show list of chains to be standardized
    2) Remove chain IDs from list of chains to be standardized
    3)

## Checking the results

1.  Notice that each chain analyzed separately, however, no structure is saved until all chains have been processed. 


2. Depending on the lenght of the sequence and the number of sequences, it may take muscle several minutes to finish an alignment. Progress will be printed to screen.


3. When a chain is first being processed, a file with chain name and '.fa' extension will be created. This file contain all the chain's unaligned sequences.


4. After each chain is processed, a file with the chain name and '.fasta' extension will be created. This file contains the MSA generated with Muscle v. 5.1. 


5. After a chain is processed, a vector with gap percentages per position is printed to screen. 


6. After ALL chains are processed and the residue number standardization is performed, a conversion template is saved in the file "OldResID_NewResID_Map.csv". Users can also save the conversion template first, without saving the renumbered structures. 


Running the cell below will display the content of "OldResID_NewResID_Map.csv" \
The file contains three columns separated by a colon: \
Old Residue ID (chain and number), New residue ID (number only), file name.



In [5]:
! cat $PROJDIR/standard_ResidueID_bank/OldResID_NewResID_Map.csv

OldResID:NewResId:File
B_3 :0 A:2y62+00.cif
B_4 :0 B:2y62+00.cif
B_5 :1:2y62+00.cif
B_6 :2:2y62+00.cif
B_7 :3:2y62+00.cif
B_8 :4:2y62+00.cif
B_9 :5:2y62+00.cif
B_10 :6:2y62+00.cif
B_11 :7:2y62+00.cif
B_12 :8:2y62+00.cif
B_13 :9:2y62+00.cif
B_14 :10:2y62+00.cif
B_15 :11:2y62+00.cif
B_16 :12:2y62+00.cif
B_17 :13:2y62+00.cif
B_18 :14:2y62+00.cif
B_19 :15:2y62+00.cif
B_20 :16:2y62+00.cif
B_21 :17:2y62+00.cif
B_22 :18:2y62+00.cif
B_23 :19:2y62+00.cif
B_24 :20:2y62+00.cif
B_25 :21:2y62+00.cif
B_26 :22:2y62+00.cif
B_27 :23:2y62+00.cif
B_28 :24:2y62+00.cif
B_29 :25:2y62+00.cif
B_30 :26:2y62+00.cif
B_31 :27:2y62+00.cif
B_32 :28:2y62+00.cif
B_33 :29:2y62+00.cif
B_34 :30:2y62+00.cif
B_35 :31:2y62+00.cif
B_36 :32:2y62+00.cif
B_37 :33:2y62+00.cif
B_38 :34:2y62+00.cif
B_39 :35:2y62+00.cif
B_40 :36:2y62+00.cif
B_41 :37:2y62+00.cif
B_42 :38:2y62+00.cif
B_43 :39:2y62+00.cif
B_44 :40:2y62+00.cif
B_45 :41:2y62+00.cif
B_46 :42:2y62+00.cif
B_47 :43:2y62+00.cif
B_48 :44:2y62+00.cif
B_49 :45:2y62+00.cif
B_50

### Congratulations you have a curated dataset! 

In the following section, we will inspect the alignments and visualize the renumbered structures. 


The  MSA saved as fasta files can be visualized with software such as [Jalview](https://www.jalview.org/). Below you can see sections of the A.fasta and B.fasta alignments which correspond to the protein chains in the TIM structures. You can refer to Step 2 for notation.

In this example, 5/6 structures contain chain A, and 6/6 structures contain chain B. 

This leads to having a different numbering system between chain A and chain B, despite TIM being a homodimer. 

For chain A, the first 2 positions are occupied by only 1 of the 6 sequences. This was also reflected in the gap percentage per position vector, where these position contains 80% gaps. However, for chain B, two sequences contain a residue at this position, accounting for a 66.6% of gaps at these positions. Which leads to the residues being counted in chain B, but not in chain A (the default threshold is 75% gap per position). 



### A.fasta

![TIM A alignment](./images/TIMJalview0_2.png)

### B.fasta

![TIM B alignment](./images/JalviewTIMB_2.png)

## Let's take a look at the structures

Having a common numbering system can facilitate comparison between structures. TIM from different organisms are different in sequence, but their structure is highly conserved. In particular, the catalytic residues in the active site are highly conserved: K 12, H 95, E 165 (based on the rabbit muscle triosephosphate isomerase)

Mapping to our newly renumbered structures, the corresponding new numbering corresponds to:

**chain A**

K 10 \
H 96 \
E 168 

**chain B**

K 10 \
H 94 \
E 166 

The image below shows a screenshoot of the renumbered structures loaded onto PyMOL. The structures were aligned to 1ag1+00.cif. 

The image shows how easy it is to label the catalytic residues in all the structures once a common residue numbering is created. 




![TIM structures hihglighting catalytic residues in sticks](./images/TIM_PyMOL_CatalyticResidues.png)

## Next Steps

For this tutorial we used a subset of all the TIM structures. We recommend repeating steps 2-4 for the whole dataset. 

Or change the keyword on step 0, to curate a dataset for a different molecule. 

